In [1]:
# %load ../../projects/misc/utils/import.py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
import datetime

import os

#Display Settings
pw = 16
ph = 9
matplotlib.rcParams['figure.figsize'] = (pw, ph)

#Pandas Dsiplay
pd.options.display.max_rows = 100
pd.options.display.max_columns = 200
pd.options.display.max_colwidth = 200

# import warnings
# warnings.filterwarnings('ignore')

In [2]:
#Constants
input_dir = '../inputs/new-york-citibike/'
output_dir = '../inputs/'

station_cols = ['start_station_id', 'start_station_name', 
                'start_station_latitude', 'start_station_longitude']

gb_cols = ['start_station_id', 'start_station_name', 'count',
           'start_station_latitude','start_station_longitude']

new_station_cols = ['station_id', 'station_name', 'count', 
                    'station_latitude', 'station_longitude']

In [3]:
#Usertype map
usertype_map = {'Subscriber': 1,
                'Customer': 2}

inv_usertype_map = {v: k for k, v in usertype_map.items()}

#Gender map
gender_map = {'unknown': 0,
              'female': 1,
              'male': 2}

inv_gender_map = {v: k for k, v in gender_map.items()}

## Stations IDs Trips vs Stations

In [47]:
#Read ids
trip_ids = pd.read_csv('../inputs/station_ids/trips_st_ids.csv')
station_ids = pd.read_csv('../inputs/station_ids/stations_st_ids.csv')

print("Stations ids in trip {} -- Station ids in stations {}".\
    format(trip_ids['station_id'].nunique(), station_ids['station_id'].nunique()))

Stations ids in trip 881 -- Station ids in stations 845


In [48]:
#Missing from stations
mfs = set(trip_ids['station_id']).difference(set(station_ids['station_id']))
len(mfs)

158

In [49]:
#Missing from trips
mft = set(station_ids['station_id']).difference(set(trip_ids['station_id']))
len(mft)

121

## Create Station ID -> Station Details Map

In [51]:
#Get stations
def get_stations(df, new_cols):
    #Most common values for station ids + count
    stations = df[station_cols].groupby('start_station_id', as_index=False)\
    .agg({'start_station_name': [lambda x: x.value_counts().index[0], 'count'],
          'start_station_latitude': lambda x: x.value_counts().index[0],
          'start_station_longitude': lambda x: x.value_counts().index[0]})
    
    stations.columns = new_cols
    return stations

In [52]:
df = pd.read_csv(input_dir + '/' + file_names[1])

#Most common values for station id
stations = get_stations(df, gb_cols)

#Find all station ids
i = 1
while stations.shape[0] < trip_ids['station_id'].nunique():
    print(i, stations.shape)
    i += 1
    
    df = pd.read_csv(input_dir + '/' + file_names[i])
    
    #Most common values for station id
    stations_t = get_stations(df, gb_cols)
    
    #Concat and keep most common
    stations = pd.concat([stations, stations_t]).sort_values('count')
    stations = stations.drop_duplicates(subset='start_station_id', keep='last')

1 (865, 5)
2 (873, 5)
3 (875, 5)
4 (876, 5)
5 (877, 5)
6 (879, 5)
7 (880, 5)
8 (880, 5)
9 (880, 5)
10 (880, 5)
11 (880, 5)
12 (880, 5)
13 (880, 5)


In [53]:
#Check if all ids are found
print('Nunique ids ', stations['start_station_id'].nunique())

Nunique ids  881


In [54]:
#Stations names with multiple ids
svc =stations['start_station_name'].value_counts() 
mt1 = svc[svc > 1]
mt1

Lafayette St & Jersey St    2
Kent Ave & N 7 St           2
Henry St & W 9 St           2
W 45 St & 6 Ave             2
Soissons Landing            2
Mercer St & Bleecker St     2
E 2 St & 2 Ave              2
Norman Ave & Leonard St     2
E 16 St & Irving Pl         2
8 Ave & W 31 St             2
Riverside Dr & W 91 St      2
Name: start_station_name, dtype: int64

In [55]:
stations[stations['start_station_name'].map(lambda x: x in mt1.index)].sort_values('start_station_name').head(6)

,start_station_id,start_station_name,count,start_station_latitude,start_station_longitude
296,521,8 Ave & W 31 St,14296,40.750450,-73.994811
503,3255,8 Ave & W 31 St,7547,40.750585,-73.994685
251,475,E 16 St & Irving Pl,6034,40.735243,-73.987586
683,3463,E 16 St & Irving Pl,2265,40.735367,-73.987974
185,403,E 2 St & 2 Ave,6518,40.725029,-73.990697
513,3265,E 2 St & 2 Ave,552,40.724563,-73.989444


**Note** Some stations were moved to a bit different location and that is why they have multiple ids for same name

In [58]:
#Rename cols
rename_dict = dict(zip(gb_cols, new_station_cols))
station = stations.rename(columns=rename_dict)

In [59]:
#Ouptu stations
stations.to_csv(output_dir + "/stations_id_map.csv")

## Compress Trips Dataset

In [4]:
#Columns to kepp in the output
keep_columns = ['tripduration', 'starttime', 'stoptime', 
                'start_station_id', 'end_station_id', 
                'usertype', 'birth_year', 'gender']

In [20]:
def compress_df(ifn, keep_columns):
    df = pd.read_csv(input_dir + '/' + ifn)
    
    #Dropna
    df = df.dropna(subset=['starttime', 'stoptime', 'start_station_id', 'end_station_id'])

    #Remove milisec
    df['starttime'] = df['starttime'].map(lambda x: x[:19]) 
    df['stoptime'] = df['stoptime'].map(lambda x: x[:19])

    #Map usertypes
    df['usertype'] = df['usertype'].map(usertype_map)
    df['usertype'].fillna(0, inplace=True)

    #Map gender
    df['gender'] = df['gender'].map(gender_map)
    df['gender'].fillna(0, inplace=True)

    df[keep_columns].to_csv(output_dir +'/' + ifn)

In [ ]:
#Compress input data
for fn in file_names[1:-1]:
    #print(fn)
    compress_df(fn, keep_columns)